# Teste Data Science Elo7 - Parte 2 - Sistema de Classificação de Produtos


## 2.1 Bibliotecas para importar

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt;
import matplotlib.gridspec as gridspec
%matplotlib inline

## 2.2 Preparação dos dados

Nesta sessão vamos apenas repetiro todo o procedimento de carregar, limpar e preparar os dados que já foi devidamente explicado no jupyter notebook referente à análise exploratória. Como tudo já está devidamente explicado em [01_analise_exploratoria.ipynb](01_analise_exploratoria.ipynb), esta etapa será feita sem as explicações e comentários entre os comandos.

In [2]:
from unidecode import unidecode

def clean_text(text):
    text = str(text)
    text = unidecode(text)
    text = text.lower()
    for t in ',.:;-_/\\+=()[]<>^~?!#*%':
        text = text.replace(t, ' ')
    return ' '.join(text.split())

def clean_and_separate_tags(text):
    text = clean_text(text)
    tags = [t for t in text.split() if not t.isdigit()]
    return tags

# carregando os dados
raw_data = pd.read_csv("elo7_recruitment_dataset.csv")
df = raw_data.copy()

# lidando com dados faltantes
df['order_counts'].fillna(0, inplace=True)
df['concatenated_tags'].fillna('', inplace=True)
df.dropna(inplace=True)
df.drop(['product_id','seller_id'], axis=1).describe()

# definindo novas variaveis
df['creation_date'] = pd.to_datetime(df['creation_date'])
date_today = pd.to_datetime('today')
df['number_of_days'] = [(date_today-day).days for day in df['creation_date']]

df['len_tags'] = [len(clean_text(t)) for t in df['concatenated_tags']]
df['len_title'] = [len(clean_text(t)) for t in df['title']]
df['len_query'] = [len(clean_text(t)) for t in df['query']]
df['price_per_weight'] = df['price']/(df['weight']+1)

df['order_probability'] = df['order_counts']/df['view_counts']
df = df[df['order_probability'] <= 1]

df['views_per_day'] = df['view_counts']/df['number_of_days']
df['orders_per_day'] = df['order_counts']/df['number_of_days']

df.drop(['product_id','seller_id'], axis=1).describe()

,search_page,position,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,number_of_days,len_tags,len_title,len_query,price_per_weight,order_probability,views_per_day,orders_per_day
count,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000,38422.000000
mean,1.491802,16.892145,84.109725,361.075191,0.780647,14.609000,546.285461,12.838998,1368.039691,45.392562,31.280126,19.922466,14.366134,0.031830,0.447680,0.010674
std,0.980284,11.589229,211.951291,1812.845573,0.413814,43.813837,1417.890272,43.288700,507.022883,47.113327,11.716516,9.339333,81.253071,0.062687,1.303772,0.036305
min,1.000000,0.000000,0.070000,0.000000,0.000000,0.000000,1.000000,0.000000,482.000000,0.000000,3.000000,1.000000,0.000034,0.000000,0.000343,0.000000
25%,1.000000,6.000000,12.760000,6.000000,1.000000,3.000000,119.000000,0.000000,1036.000000,18.000000,23.000000,13.000000,0.228834,0.000000,0.089217,0.000000
50%,1.000000,16.000000,28.520000,9.000000,1.000000,7.000000,243.000000,0.000000,1255.000000,32.000000,30.000000,19.000000,1.327889,0.000000,0.187789,0.000000
75%,2.000000,27.000000,90.000000,107.000000,1.000000,16.000000,521.000000,14.000000,1556.000000,57.000000,38.000000,25.000000,7.272601,0.042665,0.418725,0.010965
max,5.000000,38.000000,11509.380000,65009.000000,1.000000,3000.000000,45010.000000,2460.000000,4558.000000,2669.000000,60.000000,89.000000,5563.860000,1.000000,53.203310,2.318567


In [3]:
qualitative_columns = ['seller_id',
                       'product_id',
                       'creation_date',
                       'query',
                       'title',
                       'concatenated_tags',
                       'category',
                       'express_delivery']

quantitative_columns_original = list(raw_data.drop([*qualitative_columns], axis=1).columns)
quantitative_columns_new = list(df.drop([*(qualitative_columns+quantitative_columns_original)], axis=1).columns)
quantitative_columns = quantitative_columns_original + quantitative_columns_new

print('Variáveis qualitativas:')
for t in qualitative_columns:
    print(f'    - {t}')
print('\nVariáveis quantitativas do dataset original:')
for t in quantitative_columns_original:
    print(f'    - {t}')
print('\nVariáveis quantitativas criadas:')
for t in quantitative_columns_new:
    print(f'    - {t}')

Variáveis qualitativas:
    - seller_id
    - product_id
    - creation_date
    - query
    - title
    - concatenated_tags
    - category
    - express_delivery

Variáveis quantitativas do dataset original:
    - search_page
    - position
    - price
    - weight
    - minimum_quantity
    - view_counts
    - order_counts

Variáveis quantitativas criadas:
    - number_of_days
    - len_tags
    - len_title
    - len_query
    - price_per_weight
    - order_probability
    - views_per_day
    - orders_per_day
